# 0. Imports

In [1]:
!pip install pygooglenews
import pandas as pd
from pygooglenews import GoogleNews
import json
import time
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.0/362.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44951 sha256=f26f3629fe54f5e7fa4d5020d97ede97fc5d530101267dcbe56f8724476e32b2
  Stored in directory: /root/.cache/pip/wheels/67/38/07/59c89c3d334e7f1f743898af6d21c15ecb3588ad04af7ddee0
Successfully built feedparser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


#1. Mount drive

This enables us to save the scraped data to a Gdrive we have access to.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/GitHub/MNtrafficER

/content/drive/MyDrive/GitHub/MNtrafficER


In [4]:
credentials = service_account.Credentials.from_service_account_file(
    f'/content/drive/MyDrive/GitHub/MNtrafficER/key/mn-dps-incidents-03da8ed88c61.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials = credentials, project = credentials.project_id,)

# 2. Scrape news data

In [6]:
vehicles_q = '''SELECT CONCAT(name, ' ', vehicle) AS name_vehicle
FROM dps.vehicles;'''

vehicles = client.query(vehicles_q).to_dataframe()

In [8]:
current_dttm = str(datetime.now()).replace('.', '_').replace(':', '-')

In [22]:
def get_news(search_term, geo = False):
  
  time.sleep(0.25)
  gn = GoogleNews()

  if geo:
    top = gn.geo_headlines(search_term)
  
  else:
    top = gn.search(search_term)

  entries = top["entries"]

  news_df = pd.DataFrame({'search_term': search_term,
                          'geo': geo,
                          'title': [x['title'] for x in entries],
                          'published': [x['published'] for x in entries],
                          'link': [x['links'][0]['href'] for x in entries],
                          'load_dttm': datetime.now()})
  

  return news_df


In [23]:
crash_news = list(map(get_news, list(vehicles.name_vehicle)))
crash_news_df = pd.concat(crash_news)
crash_news_df['published'] = pd.to_datetime(crash_news_df['published'], infer_datetime_format = True)

# 3. Push crash news to GBQ

In [31]:
# Create Google news dataset
google_news = bigquery.Dataset('mn-dps-incidents.google_news')
google_news.location = 'US'
google_news = client.create_dataset(google_news)

# Job config
job_config = bigquery.LoadJobConfig(autodetect = True, source_format = bigquery.SourceFormat.CSV)

In [ ]:
job_config = bigquery.LoadJobConfig(
    autodetect = True,
    source_format = bigquery.SourceFormat.CSV)

In [32]:
job = client.load_table_from_dataframe(crash_news_df, 'mn-dps-incidents.google_news.crashes', job_config = job_config)

In [ ]:
# mpls_news = get_news('Minneapolis', geo = True)

# mpls_traffic = list(map(get_news, ['Minneapolis crashes', 'Minneapolis traffic', 'Twin Cities accidents', 'Twin Cities car accidents', 'Twin Cities car crashes', 'Twin Cities traffic', 'Twin Cities 212 accidents', 
#                                  'Twin Cities 394 accidents', 'Twin Cities 694 accidents', 'Twin Cities 494 accidents']))

# mpls_news_df = pd.concat([mpls_news] + mpls_traffic)

In [ ]:
# mpls_news_df.to_csv(f'data/pygooglenews_{current_dttm}.csv ')

# 4. Ping NHTSA API and push to GBQ

In [35]:
makes = pd.read_csv('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=csv')

In [40]:
# Create NHTSA dataset
nhtsa = bigquery.Dataset('mn-dps-incidents.nhtsa')
nhtsa.location = 'US'
nhtsa = client.create_dataset(nhtsa)

# Job config
job_config = bigquery.LoadJobConfig(autodetect = True, source_format = bigquery.SourceFormat.CSV)

In [41]:
job = client.load_table_from_dataframe(makes, 'mn-dps-incidents.nhtsa.vehicle_makes', job_config = job_config)

In [38]:
# honda = pd.read_csv('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/honda?format=csv')